## Object-Relational Mapper exercise
## Dorota Gawrońska-Popa


---

Using relational databases directly in programming languages other than SQL can be tedious and error-prone. One of the common approaches is to use an ORM (object-relational mapper) for that.


One of the popular ORMs for Python is SQLAlchemy.

https://www.sqlalchemy.org/

---

For the purpose of the exercise, I will create an in-memory SQLite database:

In [1]:
#  pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from decimal import Decimal
import warnings

# from sqlalchemy import and_
from sqlalchemy import func

from sqlalchemy.orm import relationship


warnings.filterwarnings("ignore")

engine = sa.create_engine('sqlite:///:memory:')
Base = declarative_base()
session = sa.orm.sessionmaker(bind=engine)()

Now, I will create three tables for the exercise:

In [4]:

class OrderItem(Base):
    __tablename__ = 'order'

    id = sa.Column('id', sa.Integer, primary_key=True)
    product_id = sa.Column('product_id', sa.Integer, sa.ForeignKey('product.id'))
    user_id = sa.Column('user_id', sa.Integer, sa.ForeignKey('user.id'))
    qty = sa.Column('qty', sa.Integer)

class Product(Base):
    __tablename__ = 'product'

    id = sa.Column('id', sa.Integer, primary_key=True)
    name = sa.Column('name', sa.Text)
    price = sa.Column('price', sa.Numeric(14,2))

class User(Base):
    __tablename__ = 'user'

    id = sa.Column('id', sa.Integer, primary_key=True)
    first_name = sa.Column('first_name', sa.Text)
    last_name = sa.Column('last_name', sa.Text)

Base.metadata.create_all(engine)

engine.echo = True



And populate their content with records:

In [5]:

session.add(User(id=42, first_name='John', last_name='Cleese'))
session.add(User(id=43, first_name='Eric', last_name='Idle'))
session.add(User(id=44, first_name='Terry', last_name='Jones'))
session.add(User(id=13, first_name='Graham', last_name='Chapman'))
session.add(Product(id=1, price=Decimal('2.10'), name='Rabbit'))
session.add(Product(id=2, price=Decimal('5.10'), name='Parrot'))
session.add(OrderItem(product_id=1, user_id=42, qty=9))
session.add(OrderItem(product_id=1, user_id=42, qty=2))
session.add(OrderItem(product_id=1, user_id=13, qty=2))
session.add(OrderItem(product_id=2, user_id=13, qty=3))
session.add(OrderItem(product_id=2, user_id=13, qty=20))
session.commit()

2021-07-28 14:27:51,901 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-28 14:27:51,904 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-28 14:27:51,904 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (1, 42, 9)
2021-07-28 14:27:51,905 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-28 14:27:51,906 INFO sqlalchemy.engine.Engine [cached since 0.002491s ago] (1, 42, 2)
2021-07-28 14:27:51,907 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-28 14:27:51,908 INFO sqlalchemy.engine.Engine [cached since 0.003991s ago] (1, 13, 2)
2021-07-28 14:27:51,908 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-28 14:27:51,909 INFO sqlalchemy.engine.Engine [cached since 0.005366s ago] (2, 13, 3)
2021-07-28 14:27:51,910 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id,

---

This is how we can pull all `User`s first names from the database:

In [6]:
session.query(User.first_name).all()

2021-07-28 14:27:51,920 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-28 14:27:51,922 INFO sqlalchemy.engine.Engine SELECT user.first_name AS user_first_name 
FROM user
2021-07-28 14:27:51,923 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ()


[('Graham',), ('John',), ('Eric',), ('Terry',)]

That corresponds roughly to a `SELECT first_name FROM user` in SQL terms.

In [7]:
session.query(User.first_name).filter(User.id > 15).all()

2021-07-28 14:27:51,937 INFO sqlalchemy.engine.Engine SELECT user.first_name AS user_first_name 
FROM user 
WHERE user.id > ?
2021-07-28 14:27:51,938 INFO sqlalchemy.engine.Engine [generated in 0.00075s] (15,)


[('John',), ('Eric',), ('Terry',)]

That corresponds roughly to a `SELECT first_name FROM user WHERE ...` in SQL terms.

---
## Exercise 1.

Taking the database from the example above, can you write ONE sqlalchemy query that will create an outer join on `Users`, `OrderItems` and `Products` that return the `last_name`, `qty` and `name` from the corresponding tables ?

The expected result should look like this:

In [8]:
# First query 
sq = session.query(User, OrderItem, Product). \
    select_from(User).outerjoin(OrderItem).join(Product).all()

for user, order, product in sq:
    print(user.last_name, order.qty, product.name)

2021-07-28 14:27:51,946 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.first_name AS user_first_name, user.last_name AS user_last_name, "order".id AS order_id, "order".product_id AS order_product_id, "order".user_id AS order_user_id, "order".qty AS order_qty, product.id AS product_id, product.name AS product_name, product.price AS product_price 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id JOIN product ON product.id = "order".product_id
2021-07-28 14:27:51,947 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()
Cleese 9 Rabbit
Cleese 2 Rabbit
Chapman 2 Rabbit
Chapman 3 Parrot
Chapman 20 Parrot


In [9]:
# ONE query
for user, order, product in session.query(User, OrderItem, Product). \
    select_from(User).outerjoin(OrderItem).join(Product).all():
    
    print(user.last_name, order.qty, product.name) 

2021-07-28 14:27:51,953 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.first_name AS user_first_name, user.last_name AS user_last_name, "order".id AS order_id, "order".product_id AS order_product_id, "order".user_id AS order_user_id, "order".qty AS order_qty, product.id AS product_id, product.name AS product_name, product.price AS product_price 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id JOIN product ON product.id = "order".product_id
2021-07-28 14:27:51,954 INFO sqlalchemy.engine.Engine [cached since 0.007942s ago] ()
Cleese 9 Rabbit
Cleese 2 Rabbit
Chapman 2 Rabbit
Chapman 3 Parrot
Chapman 20 Parrot


In [10]:
# Another way
print(session.query(User.last_name.label('User Last Name'),
                   OrderItem.qty.label('Quantity'),
                   Product.name.label('Product Name')). \
    select_from(User).outerjoin(OrderItem).outerjoin(Product))

SELECT user.last_name AS "User Last Name", "order".qty AS "Quantity", product.name AS "Product Name" 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id LEFT OUTER JOIN product ON product.id = "order".product_id


In [11]:
# And other ONE guery
print(session.query(User.last_name.label('User Last Name'),
                    OrderItem.qty.label('Quantity'),
                    Product.name.label('Product Name')) \
    .outerjoin(OrderItem, OrderItem.user_id == User.id) \
    .outerjoin(Product, Product.id == User.id))

SELECT user.last_name AS "User Last Name", "order".qty AS "Quantity", product.name AS "Product Name" 
FROM user LEFT OUTER JOIN "order" ON "order".user_id = user.id LEFT OUTER JOIN product ON product.id = user.id


---
## Exercise 2.

Taking the database from the example above, can you write ONE sqlalchemy query that will create an aggregated result from the query above (sum the products of qty and price accordingly) calculating the value of purchases (grouping) per `User` ? 

The expected result should look like this:

In [12]:
print(session.query(User.last_name.label('User Last Name'),
                    OrderItem.id.label('Order Id'),
                    Product.name.label('Product Name'),
                    Product.price.label('Product Price'),
                    func.sum(OrderItem.qty).label('Quantity Sum')) \
      .outerjoin(Product).outerjoin(User).group_by(OrderItem.user_id))

SELECT user.last_name AS "User Last Name", "order".id AS "Order Id", product.name AS "Product Name", product.price AS "Product Price", sum("order".qty) AS "Quantity Sum" 
FROM "order" LEFT OUTER JOIN product ON product.id = "order".product_id LEFT OUTER JOIN user ON user.id = "order".user_id GROUP BY "order".user_id


In [13]:
print(session.query(User.last_name.label('User Last Name'),
                    OrderItem.qty.label('Quantity'),
                    OrderItem.id.label('Order Id'),
                    Product.price.label('Product Price'),
                    Product.name.label('Product Name'),
                    func.sum(OrderItem.qty).label('Quantity Sum')). \
    select_from(User).outerjoin(OrderItem).outerjoin(Product).group_by(OrderItem.user_id))

SELECT user.last_name AS "User Last Name", "order".qty AS "Quantity", "order".id AS "Order Id", product.price AS "Product Price", product.name AS "Product Name", sum("order".qty) AS "Quantity Sum" 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id LEFT OUTER JOIN product ON product.id = "order".product_id GROUP BY "order".user_id


In [14]:
print(session.query(User.last_name.label('User Last Name'),
              User.id.label('User Id'),
              OrderItem.qty.label('Quantity'),
              OrderItem.id.label('Order Id'),
              Product.price.label('Product Price'),
              Product.name.label('Product Name'),
              func.sum(OrderItem.qty).label('Quantity Sum')) \
            .outerjoin(OrderItem, OrderItem.user_id == User.id) \
            .outerjoin(Product, Product.id == User.id).group_by(OrderItem.user_id))

SELECT user.last_name AS "User Last Name", user.id AS "User Id", "order".qty AS "Quantity", "order".id AS "Order Id", product.price AS "Product Price", product.name AS "Product Name", sum("order".qty) AS "Quantity Sum" 
FROM user LEFT OUTER JOIN "order" ON "order".user_id = user.id LEFT OUTER JOIN product ON product.id = user.id GROUP BY "order".user_id
